In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import sweetviz as sw
import seaborn as sns
sns.set()

In [2]:
#Download the dataset
shakespeare_url='https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
filepath=keras.utils.get_file('shakespeare.txt',shakespeare_url)
with open(filepath) as f:
    shakespeare_text=f.read()

In [3]:
#Tokenization
tokenizer=keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [4]:
max_id=len(tokenizer.word_index) #Number of distinct characters
dataset_size=tokenizer.document_count #Total number of characters 
[encoded]=np.array(tokenizer.texts_to_sequences([shakespeare_text]))-1 #Encoding the dataset

In [5]:
train_size=dataset_size*90//100
dataset=tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [6]:
n_steps=100
window_length=n_steps+1
dataset=dataset.repeat().window(window_length,shift=1,drop_remainder=True)

In [7]:
dataset=dataset.flat_map(lambda window: window.batch(window_length))

In [8]:
batch_size=32
dataset=dataset.shuffle(10000).batch(batch_size)
dataset=dataset.map(lambda windows: (windows[:,:-1],windows[:,1:]))
dataset=dataset.map(lambda X_batch,Y_batch: (tf.one_hot(X_batch,depth=max_id),Y_batch))
dataset=dataset.prefetch(1)

In [9]:
#Build, compile, and train the model
model=keras.models.Sequential()
model.add(keras.layers.GRU(128,return_sequences=True,input_shape=[None,max_id]))
model.add(keras.layers.GRU(128,return_sequences=True))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation='softmax')))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam')
history=model.fit(dataset,steps_per_epoch=train_size // batch_size,epochs=1)

Train for 31370 steps
31370/31370 [==============================] - 6325s 202ms/step - loss: 0.9728


In [10]:
def preprocess(texts):
    X=np.array(tokenizer.texts_to_sequences(texts))-1
    return tf.one_hot(X,max_id)

In [11]:
def next_char(text,temperature=1):
    X_new=preprocess([text])
    y_proba=model.predict(X_new)[0,-1:,:]
    rescaled_logits=tf.math.log(y_proba)/temperature
    char_id=tf.random.categorical(rescaled_logits,num_samples=1)+1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [12]:
def complete_text(text,n_chars=50,temperature=1):
    for _ in range(n_chars):
        text+=next_char(text,temperature)
    return text

In [13]:
#Generation of text
print(complete_text('t',temperature=0.2))
print(complete_text('w'))
print(complete_text('w',temperature=2))

they could not shall be set of all the off you, the
which old fellow, right workts and curelan, bid you
words chejaon hous.

iscentaturs, dy;
my dutabusood


In [14]:
print(complete_text('I',n_chars=20,temperature=5))

I her:? falsrrre-mig'


In [15]:
print(complete_text('I',n_chars=20,temperature=.5))

Imone
of greate, nobl


In [16]:
print(complete_text('I',n_chars=20,temperature=.25))

Io:
tranio, come of t
